In [ ]:
import copy
import random as rd
import csv

In [ ]:
petalType = {
    "Type 1":{
        "Usable Space":1,
        "Service Time high":300,
        "Service Time low":180,
    },
    "Type 4":{
        "Usable Space":0.5,
        "Service Time high":75,
        "Service Time low":30,
    },
    "Type 5.6":{
        "Usable Space":0.5,
        "Service Time high":90,
        "Service Time low":45,
    },
    "Type 7.8":{
        "Usable Space":0.75,
        "Service Time high":120,
        "Service Time low":60,
    },
}


path1 = {'node1':'Type 1',
'node2':'Type 5.6',
'node3':'Type 7.8',
'node4':'Type 5.6',
'node5':'Type 7.8',
'node6':'Type 4',
'node7':'Type 5.6',
'node8':'Type 5.6',
'node9':'Type 4',
'node10':'Type 5.6',
'node11':'Type 4',
'node12':'Type 7.8',
'node13':'Type 5.6',
'node14':'Type 5.6',
'node15':'Type 4',
'node16':'Type 7.8',
}

path2 = {'node17':'Type 1',
'node18':'Type 7.8',
'node19':'Type 5.6',
'node20':'Type 4',
'node21':'Type 5.6',
'node22':'Type 7.8',
'node23':'Type 4',
'node24':'Type 7.8',
'node25':'Type 4',
'node26':'Type 4',
'node27':'Type 4',
'node28':'Type 7.8',
'node29':'Type 7.8',
'node30':'Type 5.6',
'node31':'Type 4',
'node32':'Type 4',
}


path3 = {'node33':'Type 1',
'node34':'Type 4',
'node35':'Type 4',
'node36':'Type 5.6',
'node37':'Type 7.8',
'node38':'Type 5.6',
'node39':'Type 4',
'node40':'Type 4',
'node41':'Type 7.8',
'node42':'Type 4',
'node43':'Type 4',
'node44':'Type 4',
'node45':'Type 4',
'node46':'Type 4',
'node47':'Type 5.6',
'node48':'Type 4',
}

path4 = {'node8':'Type 5.6',
'node9':'Type 4',
'node10':'Type 5.6',
'node11':'Type 4',
'node12':'Type 7.8',
'node13':'Type 5.6',
'node14':'Type 5.6',
'node15':'Type 4',
'node16':'Type 7.8',}
path5 = {'node24':'Type 7.8',
'node25':'Type 4',
'node26':'Type 4',
'node27':'Type 4',
'node28':'Type 7.8',
'node29':'Type 7.8',
'node30':'Type 5.6',
'node31':'Type 4',
'node32':'Type 4',}
path6 = {'node40':'Type 4',
'node41':'Type 7.8',
'node42':'Type 4',
'node43':'Type 4',
'node44':'Type 4',
'node45':'Type 4',
'node46':'Type 4',
'node47':'Type 5.6',
'node48':'Type 4',}

# path4 = {'node24':90,'node25':30,'node26':60,'node27':60,'node28':40,'node29':20,'node30':60,'end':999}


# #Fast Track
# path10 = {'node49':90,'node27':60,'node3':60,'node57':30,'node64':30,'node36':60,'node12':60,'node51':60,'node37':40,'node13':40,
#           'node66':60,'node46':20,'node22':20,'node53':40,'node61':20,'node69':60,'end':999}

In [44]:
nodePaths = {'path1':path1,
               'path2':path2,
               'path3':path3,
                'path4':path4,
                'path5':path5,
                'path6':path6,
            #    'path4':path4,
              } #nodename : service tieme in seconds
nodeRoundTime = {
    "path1": 180, #High
    "path2": 180, #High
    "path3": 180, #High
    "path4": 180, #Low
    "path5": 180,  #Low
    "path6": 180, #Low
}
pathLiftCapacity = {
    'path1': 81, #High
    'path2': 81, #High
    'path3': 81, #High
    'path4': 9, #Low
    'path5': 9,  #Low
    'path6': 9, #Low
}
nodeDwellRange = 10 #seconds eg. 40 second dwell time with 10 second range, people can stay between 30-50 seconds

liftRoundTime = 45 #seconds
liftCapacity = 90 #people
liftCapacityArrivalRate = {
    0:1,
    3600:1
}


In [45]:
def round_down(num, divisor):
    return num - (num%divisor)

In [46]:
class person:
    def __init__(self,personIndex,timeAtNode,pathList,nodeDetail):
        self.idx = personIndex
        self.curNode = pathList[0]
        self.curNodeIdx = 0
        self.timeAtCurNode = timeAtNode
        self.nodeList = pathList
        self.node_detail = nodeDetail 
        self.timeAtNextNode = timeAtNode + self.calculateTimeToNextNode()

    def __str__(self):
        return f"{self.idx} {self.curNode} {self.timeAtCurNode} {self.timeAtNextNode}"

    def calculateTimeToNextNode(self):
        # print(self.node_detail)
        lowRange = petalType[self.node_detail[self.curNode]]['Service Time low']
        highRange = petalType[self.node_detail[self.curNode]]['Service Time high']
        timeToSpendAtCurNode = rd.randrange(lowRange,highRange+1,1)
        return timeToSpendAtCurNode

    def updateAgentLocation(self,curTime):
        if curTime == self.timeAtNextNode:
            #Reached Last Step, remove details
            if self.curNode == self.nodeList[-1]:
                self.curNode = 'end'
                self.timeAtNextNode = -1
                return
            self.curNodeIdx+=1
            self.curNode = self.nodeList[self.curNodeIdx]
            self.timeAtCurNode = curTime
            self.timeAtNextNode = curTime + self.calculateTimeToNextNode()
            return

In [49]:
#simulate arrival time of people coming in a lift and moving through from one to the next and calculate occupancy of each node over time
simulationTime = 3600 #seconds
#keep track of time when people arrive at a node, where they are and when they should leave.
numberOfPeople = 0
peopleList = []
nodeOccupancies = {}
for curTime in range(simulationTime):

    if curTime not in nodeOccupancies:
        nodeOccupancies[curTime] = {}
        for key in nodePaths:
            nodeOccupancies[curTime]['end'] = 0
            for paths in list(nodePaths[key]):
                nodeOccupancies[curTime][paths] = 0
    

    for idx,paths in enumerate(nodePaths):
        offset = idx * 12
        nodeDetails = nodePaths[paths]
        nodeList = list(nodeDetails)

        if (curTime+offset)%nodeRoundTime[paths] == 0:# People Arrive at node 1
            timeBucket = round_down(curTime, 3600)
            liftOccupancy = int(pathLiftCapacity[paths] * liftCapacityArrivalRate[timeBucket])
            for personIndex in range(liftOccupancy):
                peopleList.append(person(len(peopleList),curTime,nodeList,nodeDetails))

    for agent in peopleList:
        agent.updateAgentLocation(curTime)
        agentLocation = agent.curNode
        nodeOccupancies[curTime][agentLocation] += 1



In [50]:
# Get headers from one of the inner dictionaries
fieldnames = ['time'] + list(next(iter(nodeOccupancies.values())).keys())

with open('250424_nodeStaticCalcsOutput_Multi_petal_type.csv', 'w', newline='') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    for time, node_values in nodeOccupancies.items():
        row = {'time': time}
        row.update(node_values)
        writer.writerow(row)